In [2]:
import numpy as np
from scipy import stats
import torch
import torch.nn as nn
from tqdm import tqdm

# Longstaff-Schwartz Method with Neural Networks

This notebook implements the **Longstaff-Schwartz algorithm** using neural networks to approximate the continuation value, enabling the pricing of high-dimensional American options.

This is an extension of the classical Least-Squares Monte Carlo (LSM) method using a deep learning model to better capture non-linear relationships in the state space.

## Asset

This class simulates asset price trajectories under the **risk-neutral measure**, assuming a geometric Brownian motion (GBM) process. It supports a multi-dimensional setting with `d` assets and runs on GPU if available.

### Attributes:
- `s0` : Initial asset price (scalar or vector of size `d`).
- `r` : Risk-free interest rate.
- `div` : Continuous dividend rate.
- `sigma` : Volatility (can be a scalar or vector of shape `(d,)`).
- `T` : Time to maturity.
- `N` : Number of time steps.
- `M` : Number of Monte Carlo paths.
- `d` : Number of assets.
- `device` : `"cpu"` or `"cuda"` (or `"mps"` on Apple Silicon).

In [3]:
class Asset:
    def __init__(self, s0, r, div, sigma, T = 3, N = 9, M = 8192, d = 5, device = "cpu"):
        self.N = N
        self.M = M
        self.d = d
        self.T = T
        self.delta = T / N
        self.s0 = s0
        self.r = r
        self.div = div
        self.sigma = sigma
        self.device = device

    def brownian_paths(self, noise):
        dB = torch.zeros((noise.shape), device = self.device)
        dB[1:] = torch.sqrt(torch.tensor(self.delta, device = self.device)) * noise[1:]
        B = torch.cumsum(dB, axis = 0)

        return B

    def asset_paths(self, scenario_size = None):
        M = scenario_size or self.M

        noise = torch.randn(size = (self.N + 1, M, self.d), device = self.device)
        B = self.brownian_paths(noise)
        tn = (torch.arange(self.N + 1, device = self.device) * self.delta)[:, None, None]

        S = self.s0 * torch.exp((self.r - self.div - 0.5 * self.sigma ** 2) * tn + self.sigma * B)

        return S


## Option Classes

These classes define the **discounted payoff** for American options, based on the **maximum over `d` assets**. They support both put and call payoffs.

Each option:
- Takes an `Asset` instance.
- Defines the strike `K`.
- Implements:
  - `payoff(X, n)` to compute discounted payoff,

**Payoff formulas :**
- Put : $\max(K - \max_i S^{(i)}, 0)$  
- Call : $\max(\max_i S^{(i)} - K, 0)$

In [17]:
class MaxPutOption():
    def __init__(self, asset, K, device = "cpu"):
        self.asset = asset
        self.K = K
        self.device = device

    def payoff(self, X, n = None, discounted = True):
        max_value = torch.max(X, axis = -1)[0]
        payoff_value = torch.relu(self.K - max_value)
        if discounted :
            tn = (torch.arange(X.shape[0], device = self.device) * self.asset.delta)[:, None]
            payoff_value = torch.exp(- self.asset.r * tn) * payoff_value

        return payoff_value[n] if n is not None else payoff_value
    
class MaxCallOption():
    def __init__(self, asset, K, device = "cpu"):
        self.asset = asset
        self.K = K
        self.device = device

    def payoff(self, X, n = None, discounted = True):
        max_value = torch.max(X, axis = -1)[0]
        payoff_value = torch.relu(max_value - self.K)
        if discounted :
            tn = (torch.arange(X.shape[0], device = self.device) * self.asset.delta)[:, None]
            payoff_value = torch.exp(- self.asset.r * tn) * payoff_value

        return payoff_value[n] if n is not None else payoff_value

## Neural Network Model

A generic feedforward network with BatchNorm and ReLU activations used to approximate the continuation value at each timestep.

In [18]:
class Network(nn.Module):
    def __init__(self, input_size, layer_sizes, output_size):
        super().__init__()
        layers = []
        
        layers.append(nn.Linear(input_size, layer_sizes[0]))
        layers.append(nn.BatchNorm1d(layer_sizes[0]))
        layers.append(nn.ReLU())

        for in_size, out_size in zip(layer_sizes[:-1], layer_sizes[1:]):
            layers.append(nn.Linear(in_size, out_size))
            layers.append(nn.BatchNorm1d(out_size))
            layers.append(nn.ReLU())

        layers.append(nn.Linear(layer_sizes[-1], output_size))

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x).squeeze(-1)

## LongstaffSchwartzPricerNN

This class implements the **Longstaff-Schwartz method using neural networks** to estimate the continuation value at each time step.

Instead of a polynomial basis, this implementation uses a fully connected neural network to learn the mapping from the state (e.g., asset prices) to continuation values, making it more flexible and scalable for high-dimensional problems.

### Main methods:

- `initialize_model()` : Creates a fresh feedforward neural network (with user-defined architecture) for a given time step.
- `copy_model_weights(model_from, model_to)` : Copies weights from one neural network to another, enabling warm-start training across time steps.
- `train()` : Trains one neural network per time step in **backward order** using Monte Carlo paths:
  - For each date, the network is trained to approximate the continuation value by minimizing the squared error with respect to the future payoff.
  - Training is performed in batches using stochastic gradient descent (via the Adam optimizer).
- `compute_exercise_policy(X)` : Given trained models and new asset paths, this method applies the **optimal exercise policy** by comparing the continuation value to the immediate payoff.
- `get_price(sample_size)` : 
  - Simulates a new set of paths.
  - Applies the trained models to determine optimal stopping decisions.
  - Returns the estimated price and a confidence interval.

## Pricing Logic

### `train()` :
- For each time step `n = N-1, ..., 0`:
  - Copy the weights from the future model (`n+1`) to initialize the model at time `n`.
  - Regress the future payoff `payoff_opt` on the state `X[n]` using a neural network.
  - Update weights by minimizing the squared error over batches.
  - Once trained, use the model to determine whether early exercise is optimal:
  - Update `payoff_opt` accordingly.

### `compute_exercise_policy(X)` :
- Applies the trained policy to a new set of asset paths `X`.
- At each time step, determines whether to exercise early based on the learned continuation model.
- Returns the optimal payoff per path.

### `get_price(sample_size)` :
- Simulates a new Monte Carlo sample of asset trajectories.
- Applies the exercise policy.
- Computes the **mean payoff** as the estimated option price.
- Also returns a **confidence interval** based on the Central Limit Theorem:
  \[
  \text{CI} = \bar{V} \pm z_{\alpha/2} \cdot \frac{\sigma}{\sqrt{N}}
  \]


In [19]:
class LongstaffSchwartzPricerNN:
    def __init__(self, asset, option, layers, device = "cpu"):
        """
        Initializes a Longstaff-Schwartz pricer using a neural network model.

        Parameters:
        - asset : object representing the underlying asset.
        - option : object representing the option to be priced.
        - layers : list representing the layer sizes of the neural network.
        - device : device for execution (default is "cpu").
        """
        self.asset = asset
        self.option = option
        self.device = device
        self.layers = layers

        self.option.device = device
        self.asset.device = device

        self.thetas = None

    def initialize_model(self):
        """
        Initializes a neural network model with the appropriate input dimension.

        Returns:
        - Network : a neural network model.
        """
        input_dim = self.asset.d
        return Network(input_dim, self.layers, 1).to(self.device)

    def copy_model_weights(self, model_from, model_to):
        """
        Copies weights from one model to another.

        Parameters:
        - model_from : source model.
        - model_to : target model.
        """
        model_to.load_state_dict(model_from.state_dict())

    def train(self, n_epochs = 20, batch_size = 8192, learning_rate = 1e-3):
        """
        Trains the neural network models for each time step.

        Parameters:
        - n_epochs : number of training epochs.
        - batch_size : size of the batch for weight updates.
        - learning_rate : learning rate for the optimizer.
        
        Returns:
        - payoff_opt : estimated option value at each step.
        """
        X = self.asset.asset_paths()
        payoff = self.option.payoff(X)
        payoff_opt = payoff[-1].clone()

        self.thetas = [self.initialize_model() for _ in range(self.asset.N)]
        optimizers = [torch.optim.Adam(self.thetas[n].parameters(), lr = learning_rate) for n in range(self.asset.N)]

        for n in reversed(range(self.asset.N)):
            samples_size = X.shape[1]
            num_batches = samples_size // batch_size

            # Copy the n+1 model weight to the n model
            if n < self.asset.N - 1:
                self.copy_model_weights(self.thetas[n + 1], self.thetas[n])

            for _ in tqdm(range(n_epochs), desc = f"Training at timestep {n}"):
                indexes = torch.randperm(samples_size, device = self.device)
                for k in range(num_batches):
                    idx = indexes[k * batch_size:(k + 1) * batch_size]
                    loss = ((payoff_opt[idx] - self.thetas[n](X[n, idx]))**2).mean()
                    optimizers[n].zero_grad()
                    loss.backward()
                    optimizers[n].step()

            exercise_now = payoff[n] >= self.thetas[n](X[n])
            payoff_opt[exercise_now] = payoff[n, exercise_now].clone()

    def compute_exercise_policy(self, X):
        """
        Computes the optimal exercise policy using the trained networks, starting from the last time step.

        Parameters:
        - X : matrix of asset paths for each Monte Carlo simulation.

        Returns:
        - payoff_opt : optimal payoffs
        """
        payoff = self.option.payoff(X)
        payoff_opt = payoff[-1]

        for n in range(X.shape[0] - 2, -1, -1):
            exercise_now = payoff[n] >= self.thetas[n](X[n])
            payoff_opt[exercise_now] = payoff[n, exercise_now].clone()

        return payoff_opt

    def get_price(self, sample_size, confidence_level: float = 0.95):
        """
        Computes the estimated option price and its confidence interval.

        Parameters:
        - sample_size : number of samples for the estimation.
        - confidence_level : confidence level for the interval (default is 95%).

        Prints:
        - Estimated option price and its confidence interval.
        """
        print(f"\nPricing the option via LongStaffSchwartz NeuralNetwork...")

        X = self.asset.asset_paths(scenario_size = sample_size)
        payoff_opt = self.compute_exercise_policy(X).cpu().numpy()
        
        mean_payoff = payoff_opt.mean()
        variance = np.var(payoff_opt, ddof=1)

        alpha = 1 - confidence_level
        quantile = stats.norm.ppf(1 - alpha / 2)

        ci_lower = mean_payoff - quantile * np.sqrt(variance / sample_size)
        ci_upper = mean_payoff + quantile * np.sqrt(variance / sample_size)

        print(f"\n=== Option Pricing Results ===")
        print(f"Estimated Price     : {mean_payoff:.6f}")
        print(f"Confidence Level    : {confidence_level*100:.1f}%")
        print(f"Confidence Interval : [{ci_lower:.6f}, {ci_upper:.6f}]\n")

### Advantages of the Neural Network Approach

A key benefit of using neural networks beyond enabling pricing in higher-dimensional settings is that we do not need to manually specify a basis of functions.
Unlike traditional Longstaff-Schwartz regression, where the choice of basis (e.g., polynomials) can significantly impact accuracy, the neural network automatically learns an appropriate nonlinear representation from the data.

In [21]:
asset = Asset(s0 = 100,
              r = 0.05,
              div = 0.1,
              sigma = 0.2,
              T = 3,
              N = 9,
              M = 200_000,
              d = 3,
              device = "mps")

option = MaxCallOption(asset, 
                    K = 100, 
                    device = "mps")

pricer = LongstaffSchwartzPricerNN(asset, 
                                   option, 
                                   layers = [8, 8], 
                                   device = "mps")
pricer.train(n_epochs = 100)
pricer.get_price(sample_size = 2_960_000)

Training at timestep 0: 100%|██████████| 100/100 [00:05<00:00, 17.52it/s]



Pricing the option via LongStaffSchwartz NeuralNetwork...

=== Option Pricing Results ===
Estimated Price     : 18.604731
Confidence Level    : 95.0%
Confidence Interval : [18.585340, 18.624121]



In [22]:
option = MaxPutOption(asset, 
                    K = 100, 
                    device = "mps")

pricer = LongstaffSchwartzPricerNN(asset, 
                                   option, 
                                   layers = [8, 8], 
                                   device = "mps")
pricer.train(n_epochs = 100)
pricer.get_price(sample_size = 2_960_000)

Training at timestep 0: 100%|██████████| 100/100 [00:05<00:00, 17.77it/s]



Pricing the option via LongStaffSchwartz NeuralNetwork...

=== Option Pricing Results ===
Estimated Price     : 5.755755
Confidence Level    : 95.0%
Confidence Interval : [5.746714, 5.764796]

